Welcome to module 2.2! In the last module, we investigated how to use `scikit-learn` to build a simple Naive Bayes model to perform sentiment analysis. In this notebook we will look behind the scenes: we will build our own Naive Bayes model from scratch. We will reuse some of the code we wrote in the previous notebook - but this time, we'll write the maths from scratch.

## ❓ Pre-module quiz

Why is Naive Bayes "naive"?

A. Because it's the most basic, i.e. "naive" classifier we can build

B. Because it "naively" assumes that the probabilities of features (i.e., in our case, words) are independent of each other

C. Because the guy who invented it tought it was a cool name

D. Because it "naively" assumes that the probabilities of features (i.e., in our case, words) are dependent of each other

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>The correct answer is B - Naive Bayes assumes that the probability of finding a certain word is independent from the probability of finding another word. So, for example, in the domain of movie reviews, it assumes that the probabilities of finding the words <code>Indiana</code> and <code>Jones</code> are not correlated, even if in practice we know that this is not the case.</p>

</details> 


# Python for Computational Linguists 1.2: Breaking down Naive Bayes

## Introduction

### Recap of last notebook

Module 2.1 introduced lots of new concepts and libraries - pre-processing, feature vectors, `numpy`, and so on. In fact, we have seen how to import data from a research dataset, how to clean them by removing punctuation and stop-words, how to use `numpy` to prepare test and training data for a model, and how to use `scikit-learn` to train, test and evaluate a simple Naive Bayes classifier for sentiment analysis.

However, we didn't look 'behind the scenes' at how Naive Bayes actually works. This module will guide you through your first hand-written machine learning model, by showing you how to write the maths for Naive Bayes yourself. We will introduce the concept of "class", and we'll debug and improve our model by adding smoothing.

If you're interested, you can also just go and look the code of `scikit-learn`'s implementation [here](https://github.com/scikit-learn/scikit-learn/blob/0fb307bf3/sklearn/naive_bayes.py#L669)!

### Naive Bayes refresher

> Note: this section borrows heavily from the Naive Bayes chapter of the lecture notes. Please refer to them or to the J&M for more details.

Naive Bayes is a simple classifier based on two assumptions:
- The **bag-of-words** assumption: word ordering doesn't matter. We represent each document in our dataset as a list of pairs $(word_i,frequency_i)$.
- The **conditional independence** assumption: the probability of one word appearing in a sentence is by no means correlated to the occurrence of another word.

These two assumptions heavily simplify our model, since they completely disregard grammar and and eventual domain knowledge (e.g. the `Indiana Jones` example in the pre-module quiz); but as we know now, they allow us to build a surprisingly efficient classifier.

Let's quickly go through the maths of Naive Bayes. Remember that given a document $d$ and a set of classes $C$, the probability the document belong to a class $c \in C$ is $\hat{c}$, and is defined as:

$$ \hat{c} = \text{argmax}_{c \in C} P(c \mid d)$$

However, the Bayes Rule tells us that

$$ P(c \mid d) = \frac{P(c) \ P(d \mid c)}{P(d)} $$

Allowing us to rewrite 

$$\begin{align} 
\hat{c} &= \text{argmax}_{c \in C} P(c \mid d) \\
        &= \text{argmax}_{c \in C} \frac{P(c) \ P(d \mid c)}{P(d)}
\end{align}$$

However, the probability $d$ is constant for each class $c$, hence we can remove it, leaving only:

$$
\hat{c} = \text{argmax}_{c \in C} 
    \underbrace{P(c) \ P(d \mid c)}_\text{likelihood}
    \underbrace{P(d)}_\text{prior}
$$

Where the $prior$ is the **prior probability** of the class $c$ and the $likelihood$ is the probability of finding $d$ given the class $c$.

Using words as features, we can represent $d$ as a list of words $w_1, \dots , w_n$, hence 

$$
\hat{c} = \text{argmax}_{c \in C} 
    \underbrace{P(c) \ P(w_1, \dots , w_n \mid c)}_\text{likelihood} 
    \underbrace{P(d)}_\text{prior}
$$

However, $P(w_1, \dots , w_n \mid c)$ may be prohibitively hard to calculate, since we would need to estimate the probability of every possible combination of words. Here, the **bag-of-words** assumption comes to the rescues, assuming the probability of the words (i.e. features) are independent, allowing us to finally rewrite

$$
\begin{align}
\hat{c} &= \text{argmax}_{c \in C} 
       \underbrace{P(d)}_\text{prior}
       \underbrace{P(c) \ P(w_1 \mid c) \times \dots \times P(w_n \mid c)}_\text{likelihood} 
       \\
       &= \text{argmax}_{c \in C} 
       \underbrace{P(d)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} 
\end{align}
$$

What does all of this mean in practice? Well, that if we have a document $d$, all we need to know to classify it is:
- The *priors*, i.e. the probability of document $d$ to belong to each class $c$
- The *likelihoods*, i.e. the probabilities for each word $w_i$ of the document to belong to each class $c$.

## Naive Bayes: a simple implementation

Let's begin with a simple example from the Post Lecture exercises (taken from J&M-3, exercise 4.2). Given the following short movie reviews, each labeled with a genre, either comedy or action:

| review                      | class  |
|-----------------------------|--------|
| fun, couple, love, love     | comedy |
| fast, furious, shoot        | action |
| couple, fly, fast, fun, fun | comedy |
| curious, shoot, shoot, fun  | action |
| fly, fast, shoot, love      | action |

And a new document D: 

| review                     | class  |
|----------------------------|--------|
| fast, couple, shoot, fly   | ?      |

Compute the most likely class for D.

Let's start by saving our documents in some vectors:

In [1]:
train_docs = [
    ['fun', 'couple', 'love', 'love'],
    ['fast', 'furious', 'shoot'],
    ['couple', 'fly', 'fast', 'fun', 'fun'],
    ['curious', 'shoot', 'shoot', 'fun'],    
    ['fly', 'fast', 'shoot', 'love']]

train_labels = ['comedy', 'action', 'comedy', 'action', 'action']

### Computing the priors

Remember what we needed to do? The first step is to compute the **priors**. Let's do that with a simple function:

In [ ]:
# What are our classes?
classes = set(train_labels)
print(classes)

# initialise the priors
priors = {}
for _class in classes:
    priors[_class] = 0

# count how many train example in each class
for _class in classes:
    for label in train_labels:
        if _class == label:
            priors[_class] += 1

print(priors)

> **<h3>💻 Try it yourself!</h3>**

Now the priors are not *normalised*, i.e. we have to bring each prior in the range $[0,1]$. Can you do that in the following cell?

In [ ]:
# write your code here

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
for _class in classes:
    priors[_class] = priors[_class] / len(train_labels)
    </code></pre></p>

</details> 

If you've got the priors correct, you should have $P(comedy) = 0.4$ and $P(action)=0.6$.

In [ ]:
print(priors)

> **<h3>💻 Try it yourself!</h3>**

Now let's wrap everything nicely into a function. Can you complete the cell below?

In [ ]:
def compute_priors(labels):
    '''
    Computes the priors for a set of labels.
    '''

    # What are our classes?
    classes = set(labels)

    # initialise the priors
    priors = {}

    # ...?

    # count how many train example in each class
    
    # ...?
    
    # normalise the priors
    
    # ...?
    
    return priors

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def compute_priors(labels):
    '''
    Computes the priors for a set of labels.
    '''
    
    # What are our classes?
    classes = set(labels)

    # initialise the priors
    priors = {}
    for _class in classes:
        priors[_class] = 0

    # count how many train example in each class
    for _class in classes:
        for label in labels:
            if _class == label:
                priors[_class] += 1
    
    # normalise the priors
    for _class in classes:
        priors[_class] = priors[_class] / len(labels)
        
    return priors
    </code></pre></p>

</details> 

In [ ]:
priors = compute_priors(train_labels)
print(priors)

### Computing the likelihoods

Now we need to compute the likelihoods of the words w.r.t. to each class. To do that, we can build a `dict` for each class, where `dict[word] = P(word|class)`. To do that we can slightly modify the function `create_vocab_dict` that we defined in [Module 1.4](../../module_1/module_1.4/module_1.4.ipynb). The function we defined was the following:

In [ ]:
def create_vocab_dict(lines):
    '''
    Collect vocabulary counts from text

    Parameters
    ----------
    f_processed_arg: a list of list of words

    Returns
    -------
    a dictionary with words (str) as keys and counts(int) as values
    vocab={
    'SONNETS': 1
    }
    '''
    vocab={}# create an empty vocabulary dictionary to store words as keys and counts as values later. 
    for line in lines:
        for word in line:
            if word in vocab:
                vocab[word]+=1 # update the count for an existing word
            else:
                vocab[word]=1 # initilize the count for a new word
    return vocab

In [ ]:
print(create_vocab_dict(train_docs))

> **<h3>💻 Try it yourself!</h3>**

How can we modify this function to give us the likelihoods for each class? Modify the function below, directly derived from `create_vocab_dict`, to return the likelihoods instead of the raw counts.

In [22]:
def compute_likelihoods(lines):
    '''
    Computes the likelihoods of words in a list of strings.

    Parameters
    ----------
    f_processed_arg: a list of list of words

    Returns
    -------
    a dictionary with words (str) as keys and likelihoods(floats) as values
    vocab={
    'SONNETS': 0.01
    }
    '''
    vocab={}# create an empty vocabulary dictionary to store words as keys and counts as values later. 
    for line in lines:
        for word in line:
            if word in vocab:
                vocab[word]+=1 # update the count for an existing word
            else:
                vocab[word]=1 # initilize the count for a new word

    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # how long are our documents?
    total_tokens = 0
    
    # write your code here
    # (hint: sum the length of the lines in total_tokens!)
    
    for word in vocab:
        vocab[word] = vocab[word]/total_tokens

    return vocab

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def compute_likelihoods(lines):
    '''
    Computes the likelihoods of words in a list of strings.

    Parameters
    ----------
    f_processed_arg: a list of list of words

    Returns
    -------
    a dictionary with words (str) as keys and likelihoods(floats) as values
    vocab={
    'SONNETS': 0.01
    }
    '''
    vocab={}# create an empty vocabulary dictionary to store words as keys and counts as values later. 
    for line in lines:
        for word in line:
            if word in vocab:
                vocab[word]+=1 # update the count for an existing word
            else:
                vocab[word]=1 # initilize the count for a new word

    # how long are our documents?
    total_tokens = 0
    for line in lines:
        total_tokens += len(line)    
    
    for word in vocab:
        vocab[word] = vocab[word]/total_tokens

    return vocab
    </code></pre></p>

</details> 

In [25]:
print(compute_likelihoods(train_docs))

{'fun': 0.2, 'couple': 0.1, 'love': 0.15, 'fast': 0.15, 'furious': 0.05, 'shoot': 0.2, 'fly': 0.1, 'curious': 0.05}


Note that this method computes the likelihoods of the words in the whole vocabulary, irregardless of the classes. To get the likelihoods for each class we need to do as follows:

In [26]:
target_class = 'action'
target_docs = []

# enumerate builds an (index, doc) list, hence allowing
# us to retrieve the label for each doc
for i, doc in enumerate(train_docs):
    if train_labels[i] == target_class:
        target_docs.append(doc)
        
print(target_docs)
print(compute_likelihoods(target_docs))

[['fast', 'furious', 'shoot'], ['curious', 'shoot', 'shoot', 'fun'], ['fly', 'fast', 'shoot', 'love']]
{'fast': 0.18181818181818182, 'furious': 0.09090909090909091, 'shoot': 0.36363636363636365, 'curious': 0.09090909090909091, 'fun': 0.09090909090909091, 'fly': 0.09090909090909091, 'love': 0.09090909090909091}


### The training function

> **<h3>💻 Try it yourself!</h3>**

Now we have all the instruments to build our training function. Can you complete it?

In [33]:
def train_naive_bayes(documents, labels):
    
    classes = set(labels)
    
    # compute the priors
    priors = compute_priors(labels)
    
    # this dict will contain the likelihoods, e.g.
    # likelihoods['action'] = {'fast': 0.2, 'furious': 0.1...
    likelihoods = {}
    
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    for _class in classes:
        # get the documents belonging to each class:
        class_docs = []

        # put your code here
        for #...
        
        # compute the likelihood of this class
        likelihoods[_class] = # hint: use compute_likekihood defined above
        
    return priors, likelihoods    

SyntaxError: invalid syntax (<ipython-input-33-3675c1ff6099>, line 20)

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def train_naive_bayes(documents, labels):
    
    classes = set(labels)
    
    # compute the priors
    priors = compute_priors(labels)
    
    # this dict will contain the likelihoods, e.g.
    # likelihoods['action'] = {'fast': 0.2, 'furious': 0.1...
    likelihoods = {}
    
    for _class in classes:
        # get the documents belonging to each class:
        class_docs = []

        for i, doc in enumerate(documents):
            if labels[i] == _class:
                class_docs.append(doc)
        
        # compute the likelihood of this class
        likelihoods[_class] = compute_likelihoods(class_docs)
        
    return priors, likelihoods 
    </code></pre></p>

</details> 

In [35]:
priors, likelihoods = train_naive_bayes(train_docs, train_labels)

print('Priors:')
print(priors)

print('')
print('Likelihoods:')
print(likelihoods)

Priors:
{'action': 0.6, 'comedy': 0.4}

Likelihoods:
{'action': {'fast': 0.18181818181818182, 'furious': 0.09090909090909091, 'shoot': 0.36363636363636365, 'curious': 0.09090909090909091, 'fun': 0.09090909090909091, 'fly': 0.09090909090909091, 'love': 0.09090909090909091}, 'comedy': {'fun': 0.3333333333333333, 'couple': 0.2222222222222222, 'love': 0.2222222222222222, 'fly': 0.1111111111111111, 'fast': 0.1111111111111111}}


### Predict unknown classes

So now we have trained our model. How can we predict the likelihood of new sentences belonging to each class? Remember from above that

$$
\hat{c} = \text{argmax}_{c \in C} 
       \underbrace{P(d)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} 
$$

So, to predict the class of our new sentence `fast, couple, shoot, fly`, we need to:
- for each class `c`, we need to calculate `prob_c` by
    - multiplying the probability of each word `w` given that class `c`

Then, select the maximum of our `prob_c`s.

> **<h3>💻 Try it yourself!</h3>**

Can you predict the class of `fast, couple, shoot, fly`.

In [39]:
document = ['fast', 'couple', 'shoot', 'fly']

# -------------------------#
#      E X E R C I S E     #
# -------------------------#

# Please put the calculated probabilities in this dictionary
# e.g. document_probabilities['action'] = ..?
document_probabilities = {}

# Write your code here:
for label in train_labels:
    prob_prior = priors[label]
    prob_class = prob_prior
    for word in document:
        if word in likelihoods[label]:
            prob_class = prob_prior*likelihoods[label][word]
        
    document_probabilities[label] = prob_class

In [40]:
print(document_probabilities)

{'comedy': 0.044444444444444446, 'action': 0.05454545454545454}


<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def train_naive_bayes(documents, labels):
    
    classes = set(labels)
    
    # compute the priors
    priors = compute_priors(labels)
    
    # this dict will contain the likelihoods, e.g.
    # likelihoods['action'] = {'fast': 0.2, 'furious': 0.1...
    likelihoods = {}
    
    for _class in classes:
        # get the documents belonging to each class:
        class_docs = []

        for i, doc in enumerate(documents):
            if labels[i] == _class:
                class_docs.append(doc)
        
        # compute the likelihood of this class
        likelihoods[_class] = compute_likelihoods(class_docs)
        
    return priors, likelihoods 
    </code></pre></p>

</details> 

## Object-oriented programming in a nutshell

### Your first class: `Dataset`

### The `NaiveBayes` class

### Test Naive Bayes on new data

### Adding 1-smoothing

## Re-classify the *Thumbs up?* dataset

### Recap of loader functions

### Applying our model

### Improving the results

### Differences between our model and `scikit-learn`

## Wrapping up